In [138]:
import pandas as pd
import json
import os
import glob
import re

In [139]:
def get_column_names(schema,data_set):
    col_details = sorted(schema[data_set],key= lambda i : i['column_position'])
    return [i['column_name'] for i in col_details]

In [140]:
def get_file_names(params):
    source_path_regex = ''

    if params['source_entity']=='all':
        source_path_regex= params['source_file_path']+'\\*\\part-*'
    else:
        source_path_regex= params['source_file_path'] + '\\' + params['source_entity'] + '\\part-*'

    return glob.glob(source_path_regex,recursive=True)

In [141]:
def get_ds_names(file_names):
    ds_name = set()
    for file in file_names:
        ds_name.add(file.split('\\')[2])
    return ds_name

In [142]:
#variables
param_path='params.json'

params = json.load(open(param_path))
schema = json.load(open(params['schema_details']))

file_names = get_file_names(params)

print(file_names)

['data\\retail_db\\orders\\part-00000']


In [143]:
col_names = get_column_names(schema,'orders')

In [144]:
for file in file_names:
    df = pd.read_csv(file,names=col_names)
    print(f'shape of {file} is {df.shape}')

shape of data\retail_db\orders\part-00000 is (68883, 4)


In [145]:
ds_names = list(get_ds_names(file_names))
print(ds_names)

['orders']


In [149]:
for file in file_names:
    print(f'Processing file {file}')
    ds = file.split('\\')[2]
    file_nm = file.split('\\')[-1]
    col_names = get_column_names(schema,ds)
    df = pd.read_csv(file,names=col_names)
    print(f'Shape of {file} is {df.shape}')

    #Setting target file path
    target_file_path = params['target_file_path'] + '\\' + ds + '\\'+ file_nm + '.' + params['target_format']
    print(f'converted file\'s relative path will be {target_file_path}')

    print(f'Creating path {params['target_file_path']+'\\'+ds} if it doesn\'t exist')
    os.makedirs(params['target_file_path'] + '\\' + ds, exist_ok=True)
    print(f'Created path {params['target_file_path']+'\\'+ds} if it doesn\'t exist')

    #Writing into desired target file
    if params['target_format']=='json':
        print(f'Copying from {file} into {target_file_path}')
        df.to_json(target_file_path)
        print(f'Copied from {file} into {target_file_path}')
    elif params['target_format']=='csv':
        print(f'Copying from {file} into {target_file_path}')
        df.to_csv(target_file_path)
        print(f'Copied from {file} into {target_file_path}')
    else:
        print(f'Invalid target_format {params['target_format']} is provided in params.json')
    
    print(f'{file} is processed and it\'s {params['target_format']} converted file is saved as {target_file_path}')


Processing file data\retail_db\orders\part-00000
Shape of data\retail_db\orders\part-00000 is (68883, 4)
converted file's relative path will be converted_files\retail_db\orders\part-00000.json
Creating path converted_files\retail_db\orders if it doesn't exist
Created path converted_files\retail_db\orders if it doesn't exist
Copying from data\retail_db\orders\part-00000 into converted_files\retail_db\orders\part-00000.json
Copied from data\retail_db\orders\part-00000 into converted_files\retail_db\orders\part-00000.json
data\retail_db\orders\part-00000 is processed and it's json converted file is saved as converted_files\retail_db\orders\part-00000.json
